In [ ]:
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam

In [2]:
from ImageLoader import ImageLoader
import os

image_dir = "images/"
image_paths = [os.path.join(image_dir, "pictures", fname) for fname in os.listdir(os.path.join(image_dir, "pictures"))]
labels_paths = [os.path.join(image_dir, "manual", fname) for fname in os.listdir(os.path.join(image_dir, "manual"))]
masks_paths = [os.path.join(image_dir, "mask", fname) for fname in os.listdir(os.path.join(image_dir, "mask"))]

images = ImageLoader.load_images(image_paths)
labels = ImageLoader.load_images(labels_paths)
masks =ImageLoader.load_images(masks_paths)

In [ ]:
print(images[0].shape, labels[0].shape, masks[0].shape)
print(len(images), len(labels), len(masks))

(2336, 3504, 3) (2336, 3504) (2336, 3504, 3)
42 42 42
0 1


In [ ]:
from PatchFeatureExtractor import PatchFeatureExtractor

pfe = PatchFeatureExtractor()

features = [] 

pfe.extract_features(images[0], labels[0], masks[0], csv_path="datasets/features.csv")

# for i in range(len(images)):
#     features.append(pfe.extract_features(images[i], labels[i], masks[i]))

Number of valid patch centers: 6834668


In [ ]:
df = pd.read_csv("datasets/features.csv")

In [23]:
count_0 = (df['label'] == 0).sum()
count_255 = (df['label'] != 0).sum()
print("label == 0:", count_0)
print("label == 255:", count_255)

# NIE ZGADZA SIĘ COŚ Z N FEATURES

label == 0: 5000
label == 255: 5000


In [ ]:
y, x = features

In [6]:
print(len(y), len(x))
#batch size x channels x num features

5000 5000


In [7]:
from Dataset import Dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=5, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=20, random_state=1)

train_data = Dataset(X_train, y_train)
valid_data = Dataset(X_val, y_val)
test_data = Dataset(X_test, y_test)

In [ ]:
batch_size = 256

train_loader = DataLoader(train_data, batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size)
test_loader = DataLoader(test_data, batch_size)

In [ ]:
h_size = 512
n_classes = 2

model 

loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

TypeError: descriptor '__init__' of 'super' object needs an argument

In [ ]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [ ]:
def validate(model):
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [ ]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [ ]:
    epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()